# Convolutional Neural Networks

This notebook introduces convolutional neural networks (CNNs), a more powerful classification model similar to the Neural Bag-of-Words (BOW) model you explored earlier.

## Outline

- **Part (a):** Model Architecture
- **Part (b):** Implementing the CNN Model
- **Part (c):** Tuning

In [1]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io
# Code for this assignment
import sst

# Monkey-patch NLTK with better Tree display that works on Cloud or other display-less server.
print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=sst.sst_node_style, format='svg')

Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.


## (a) Model Architecture

CNNs are a more sophisticated neural model for sentence classification than the Neural BOW model we saw in the last section. CNNs operate by sweeping a collection of filters over a text. Each filter produces a sequence of feature values known as a _feature map_. In one of the most basic formulations introduced by [Kim (2014)](https://www.aclweb.org/anthology/D14-1181), a single layer of _pooling_ is used to summarize the _feature maps_ as a fixed length vector. The fixed length vector is then feed to the output layer in order to produce classification labels. A popular choice for the pooling operation is to take the maximum feature value from by each _feature map_.

![Convolutional Neural Network from Kim 2014](kim_2014_figure_1_cnn.png)
*CNN model architure, Figure 1 from Kim (2014)*

We'll use the following notation:
- $w^{(i)} \in \mathbb{Z}$, the word id for the $i^{th}$ word of the sequence (as an integer index)
- $x^{(i)} \in \mathbb{R}^d$ for the vector representation (embedding) of $w^{(i)}$
- $x^{i:i+j}$ is the concatenation of $x^{(i)}, x^{(i+1)} ... x^{(i+j)}$ 
- $c^{(i)}_{k}$ is the value of the $k^{th}$ feature map along the word sequence, each filter applies over a window of $h$ words and uses non-linearity $f$.
- $\hat{c}_{k}$ is the value of the $k^{th}$ feature after pooling the feature map over the whole sequence.
- $\hat{C}$ is the concatenation of pooled feature maps. 
- $y$ for the target label ($\in 1,\ldots,\mathtt{num\_classes}$)

Our model is defined as:
- **Embedding layer:** $x^{(i)} = W_{embed}[w^{(i)}]$
- **Convolutional layer:** $c^{(i)}_{k} = f(x^{i:i+h-1} W_k + b)$
- **Pooling layer:**  $\hat{c}_{k}$ = $max(c^{(0)}_{k}, c^{(1)}_{k}...)$ 
- **Output layer:** $\hat{y} = \hat{P}(y) = \mathrm{softmax}(\hat{C} W_{out} + b_{out})$


We'll refer to the first part of this model (**Embedding layer**, **Convolutional layer**, and **Pooling layer**) as the **Encoder**: it has the role of encoding the input sequence into a fixed-length vector representation that we pass to the output layer.

We'll also use these as shorthand for important dimensions:
- `V`: the vocabulary size (equal to `ds.vocab.size`)
- `N`: the maximum number of tokens in the input text
- `embed_dim`: the embedding dimension $d$
- `kernel_size`: a list of filter lengths
- `filters`: number filters per filter length
- `num_classes`: the number of target classes (2 for the binary task)

## (a) Short Answer Questions

When answering these questions in the answers file,
`embed_dim = 10`, `kernel_size = [3, 4, 5]`, `filters=128`, `N=10` and `num_classes = 7`.

1. In terms of these values, the vocabulary size `V` and the maximum sequence length `N`, what are the
   shapes of the following variables: 
   $c^{(i)}_{kernal\_size=3}$, $c^{(i)}_{kernal\_size=4}$, $c^{(i)}_{kernal\_size=5}$, $\hat{c}^{(i)}_{kernal\_size=3}$, $\hat{c}^{(i)}_{kernal\_size=4}$, $\hat{c}^{(i)}_{kernal\_size=5}$, and $\hat{C}$. Assume a stride size of 1. Assume padding is not used (e.g., for tf.nn.max_pool and tf.nn.conv1d, setting padding='VALID'), provide the shapes listed above.
<p>
2. What are the shapes of $c^{(i)}_{kernal\_size=3}$ and $\hat{c}^{(i)}_{kernal\_size=3}$ when paddiding is used.
      (e.g., for tf.nn.max_pool and tf.nn.conv1d, setting padding='same').
<p>
3. How many parameters are in each of the convolutional filters, $W_{filter\_length=3}$, $W_{filter\_length=4}$, $W_{filter\_length=5}$? And the output layer, $W_{out}$?
<p>
<p>
4. Historically NLP models made heavy use of manual feature engineering. In relation to systems with manually engineered features, describe what type of operation is being performed by the convolutional filters.
<p>
5. Suppose that we have two examples, `[foo bar baz]` and `[baz bar foo]`. Will this model make the same predictions on these? Why or why not?

## (b) Implementing the CNN Model

We'll implement our CNN model below. Our implementation will differ from [Kim (2014)](https://www.aclweb.org/anthology/D14-1181) in that we will support using multiple dense hidden layers after the convolutional layers.

**Before you start**, be sure to answer the short-answer questions above!

In [2]:
import sst

# Load SST dataset
ds = sst.SSTDataset(V=20000).process(label_scheme="binary")
max_len = 40
train_x, train_ns, train_y = ds.as_padded_array('train', max_len=max_len, root_only=True)
dev_x,   dev_ns,   dev_y   = ds.as_padded_array('dev',   max_len=max_len, root_only=True)
test_x,  test_ns,  test_y  = ds.as_padded_array('test',  max_len=max_len, root_only=True)


Loading SST from data/sst/trainDevTestTrees_PTB.zip
Training set:     8,544 trees
Development set:  1,101 trees
Test set:         2,210 trees
Building vocabulary - 16,474 words
Processing to phrases...  Done!
Splits: train / dev / test : 98,794 / 13,142 / 26,052


In [27]:
# Specify model hyperparameters.

def run(
    epochs=10, embed_dim=5, num_filters=[2,2,2], kernel_sizes=[2,3,4],
    dense_layer_dims=[512, 256], dropout=0.4
):
    num_classes = len(ds.target_names)

    # Construct the convolutional neural network.
    # The form of each keras layer function is as follows:
    #    result = keras.layers.LayerType(arguments for the layer)(layer(s) it should use as input)
    # concretely,
    #    this_layer_output = keras.layers.Dense(100, activation='relu')(prev_layer_vector)
    # performs this_layer_output = relu(prev_layer_vector x W + b) where W has 100 columns.

    # Input is a special "layer".  It defines a placeholder that will be overwritten by the training data.
    # In our case, we are accepting a list of wordids (padded out to max_len).
    wordids = keras.layers.Input(shape=(max_len,))

    # Embed the wordids.
    # Recall, this is just a mathematically equivalent operation to a linear layer and a one-hot
    h = keras.layers.Embedding(ds.vocab.size, embed_dim, input_length=max_len)(wordids)

    # Construct "filters" randomly initialized filters with dimension "kernel_size" for each size of filter we want.
    # With the default hyperparameters, we construct 10 filters each of size 2, 3, 4.  As in the image above, each filter
    # is wide enough to span the whole word embedding (this is why the convolution is "1d" as seen in the
    # function name below).
    conv_layers_for_all_kernel_sizes = []
    for filters, kernel_size in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)

    # Concat the feature maps from each different size.
    h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)

    # Dropout can help with overfitting (improve generalization) by randomly 0-ing different subsets of values
    # in the vector.
    # See https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf for details.
    h = keras.layers.Dropout(rate=dropout_rate)(h)

    ### YOUR CODE HERE
    # Add a fully connected layer for each dense layer dimension in dense_layer_dims.
    for dim in dense_layer_dims:
        h = keras.layers.Dense(dim, activation='relu')(h)
    ### END YOUR CODE

    prediction = keras.layers.Dense(num_classes, activation='softmax')(h)

    model = keras.Model(inputs=wordids, outputs=prediction)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  # From information theory notebooks.
                  metrics=['accuracy'])        # What metric to output as we train.
    
    print({
        'epochs': epochs, 'embed_dim': embed_dim, 'num_filters': num_filters, 'kernel_sizes': kernel_sizes,
        'dense_layer_dims': dense_layer_dims, 'dropout': dropout
    })
    model.reset_states()
    model.fit(train_x, train_y, epochs=epochs)
    
    return model

In [20]:
model.reset_states()
model.fit(train_x, train_y, epochs=epochs)

Epoch 1/10
6920/6920 [==============================] - 2s 338us/sample - loss: 0.6911 - acc: 0.5185
Epoch 2/10
6920/6920 [==============================] - 2s 292us/sample - loss: 0.6269 - acc: 0.6461
Epoch 3/10
6920/6920 [==============================] - 2s 294us/sample - loss: 0.5008 - acc: 0.7642
Epoch 4/10
6920/6920 [==============================] - 2s 293us/sample - loss: 0.3958 - acc: 0.8259
Epoch 5/10
6920/6920 [==============================] - 2s 288us/sample - loss: 0.3410 - acc: 0.8585
Epoch 6/10
6920/6920 [==============================] - 2s 291us/sample - loss: 0.2881 - acc: 0.8808
Epoch 7/10
6920/6920 [==============================] - 2s 295us/sample - loss: 0.2555 - acc: 0.8967
Epoch 8/10
6920/6920 [==============================] - 2s 290us/sample - loss: 0.2263 - acc: 0.9100
Epoch 9/10
6920/6920 [==============================] - 2s 302us/sample - loss: 0.2124 - acc: 0.9169
Epoch 10/10
6920/6920 [==============================] - 2s 287us/sample - loss: 0.1965 - a

## Evaluation

Call [evaluate](https://keras.io/models/model/#evaluate) on your model.

In [21]:
#### YOUR CODE HERE ####
model.evaluate(dev_x, dev_y)
#### END(YOUR CODE) ####

872/872 [==============================] - 0s 214us/sample - loss: 0.8311 - acc: 0.6950


[0.8311196215655825, 0.69495416]

# Part (c): Tuning Your Model

We'll once again want to optimize hyperparameters for our model to see if we can improve performance. The CNN model includes a number of new parameters that can significantly influence model performance.

In this section, you will be asked to describe the new parameters as well as use them to attempt to improve the performance of your model.

## Part (c) Short Answer Questions

  1. Choose two parameters unique the CNN model, perform at least 10 runs with different combinations of values for these parameters, and then report the dev set results below. ***Hint: Consider wrapping the training code above in a for loop the examines the different values.***  To do this efficiently, you should consider [this paper](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf) from Bergstra and Bengio.  [This blog post](https://blog.floydhub.com/guide-to-hyperparameters-search-for-deep-learning-models/) also has a less formal treatment of the same topic.
  2. Describe any trends you see in experiments above (e.g., can you identify good ranges for the individual parameters; are there any interesting interactions?)
  3. Pick the three best configurations according to the dev set and evaluate them on the test data. Is the ranking of the three best models the same on the dev and test sets?
  4. What was the best accuracy you achieved on the test set?

In [28]:
model1 = run(embed_dim=50)

{'epochs': 10, 'embed_dim': 50, 'num_filters': [2, 2, 2], 'kernel_sizes': [2, 3, 4], 'dense_layer_dims': [512, 256], 'dropout': 0.4}
Epoch 1/10
6920/6920 [==============================] - 5s 704us/sample - loss: 0.6718 - acc: 0.5678
Epoch 2/10
6920/6920 [==============================] - 4s 642us/sample - loss: 0.5332 - acc: 0.7328
Epoch 3/10
6920/6920 [==============================] - 4s 633us/sample - loss: 0.3751 - acc: 0.8373
Epoch 4/10
6920/6920 [==============================] - 5s 652us/sample - loss: 0.2838 - acc: 0.8796
Epoch 5/10
6920/6920 [==============================] - 5s 657us/sample - loss: 0.2356 - acc: 0.9014
Epoch 6/10
6920/6920 [==============================] - 5s 652us/sample - loss: 0.1923 - acc: 0.9208
Epoch 7/10
6920/6920 [==============================] - 4s 647us/sample - loss: 0.1597 - acc: 0.9360
Epoch 8/10
6920/6920 [==============================] - 4s 644us/sample - loss: 0.1451 - acc: 0.9422
Epoch 9/10
6920/6920 [==============================] - 4s 

[0.6158494138662968, 0.7465596]

In [29]:
model1.evaluate(dev_x, dev_y)

872/872 [==============================] - 0s 74us/sample - loss: 0.6158 - acc: 0.7466


[0.6158494138662968, 0.7465596]

In [30]:
model2 = run(embed_dim=100, dropout=0.7, dense_layer_dims=[256, 256])

{'epochs': 10, 'embed_dim': 100, 'num_filters': [2, 2, 2], 'kernel_sizes': [2, 3, 4], 'dense_layer_dims': [256, 256], 'dropout': 0.7}
Epoch 1/10
6920/6920 [==============================] - 9s 1ms/sample - loss: 0.6597 - acc: 0.5874
Epoch 2/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.5006 - acc: 0.7616
Epoch 3/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.3580 - acc: 0.8412
Epoch 4/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.2648 - acc: 0.8882
Epoch 5/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.2138 - acc: 0.9108
Epoch 6/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.1746 - acc: 0.9253
Epoch 7/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.1457 - acc: 0.9389
Epoch 8/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.1166 - acc: 0.9526
Epoch 9/10
6920/6920 [==============================] - 8s 1ms/sample - lo

In [31]:
model2.evaluate(dev_x, dev_y)

872/872 [==============================] - 0s 321us/sample - loss: 0.7096 - acc: 0.7294


[0.7096204292883567, 0.7293578]

In [32]:
model3 = run(embed_dim=100, dropout=0.7, dense_layer_dims=[128, 128], num_filters=[2,2,2], kernel_sizes=[2,4,8])

{'epochs': 10, 'embed_dim': 100, 'num_filters': [2, 2, 2], 'kernel_sizes': [2, 4, 8], 'dense_layer_dims': [128, 128], 'dropout': 0.7}
Epoch 1/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.6600 - acc: 0.5957
Epoch 2/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.4713 - acc: 0.7887
Epoch 3/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.3055 - acc: 0.8767
Epoch 4/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.2059 - acc: 0.9201
Epoch 5/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.1404 - acc: 0.9457
Epoch 6/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.1249 - acc: 0.9514
Epoch 7/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.0950 - acc: 0.9629
Epoch 8/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.0830 - acc: 0.9684
Epoch 9/10
6920/6920 [==============================] - 8s 1ms/sample - lo

In [33]:
model3.evaluate(dev_x, dev_y)

872/872 [==============================] - 0s 349us/sample - loss: 0.9191 - acc: 0.7466


[0.9191371869603429, 0.7465596]

In [34]:
model4 = run(embed_dim=100, dropout=0.8, dense_layer_dims=[64, 32], num_filters=[2,2,2], kernel_sizes=[2,4,8])

{'epochs': 10, 'embed_dim': 100, 'num_filters': [2, 2, 2], 'kernel_sizes': [2, 4, 8], 'dense_layer_dims': [64, 32], 'dropout': 0.8}
Epoch 1/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.6751 - acc: 0.5634
Epoch 2/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.5195 - acc: 0.7486
Epoch 3/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.3519 - acc: 0.8551
Epoch 4/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.2363 - acc: 0.9051
Epoch 5/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.1693 - acc: 0.9360
Epoch 6/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.1223 - acc: 0.9548
Epoch 7/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.1043 - acc: 0.9597
Epoch 8/10
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.0836 - acc: 0.9694
Epoch 9/10
6920/6920 [==============================] - 8s 1ms/sample - loss

In [35]:
model4.evaluate(dev_x, dev_y)

872/872 [==============================] - 0s 363us/sample - loss: 0.9118 - acc: 0.7500


[0.9118259844429996, 0.75]

In [44]:
model5 = run(epochs=5, embed_dim=100, dropout=0.8, dense_layer_dims=[64, 32], num_filters=[2,2,2], kernel_sizes=[2,4,8])

{'epochs': 5, 'embed_dim': 100, 'num_filters': [2, 2, 2], 'kernel_sizes': [2, 4, 8], 'dense_layer_dims': [64, 32], 'dropout': 0.8}
Epoch 1/5
6920/6920 [==============================] - 9s 1ms/sample - loss: 0.6591 - acc: 0.6000
Epoch 2/5
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.4860 - acc: 0.7785
Epoch 3/5
6920/6920 [==============================] - 9s 1ms/sample - loss: 0.3219 - acc: 0.8686
Epoch 4/5
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.2168 - acc: 0.9160
Epoch 5/5
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.1577 - acc: 0.9358


In [45]:
model5.evaluate(dev_x, dev_y)

872/872 [==============================] - 0s 425us/sample - loss: 0.6351 - acc: 0.7569


[0.6350961896257663, 0.75688076]

In [38]:
model6 = run(epochs=5, embed_dim=50, dropout=0.7, dense_layer_dims=[128, 64], num_filters=[2,2,2], kernel_sizes=[2,4,6])

{'epochs': 5, 'embed_dim': 50, 'num_filters': [2, 2, 2], 'kernel_sizes': [2, 4, 6], 'dense_layer_dims': [128, 64], 'dropout': 0.7}
Epoch 1/5
6920/6920 [==============================] - 5s 714us/sample - loss: 0.6745 - acc: 0.5595
Epoch 2/5
6920/6920 [==============================] - 4s 650us/sample - loss: 0.5235 - acc: 0.7462
Epoch 3/5
6920/6920 [==============================] - 4s 649us/sample - loss: 0.3438 - acc: 0.8514
Epoch 4/5
6920/6920 [==============================] - 4s 649us/sample - loss: 0.2427 - acc: 0.9036
Epoch 5/5
6920/6920 [==============================] - 5s 653us/sample - loss: 0.1772 - acc: 0.9295


In [39]:
model6.evaluate(dev_x, dev_y)

872/872 [==============================] - 0s 355us/sample - loss: 0.6064 - acc: 0.7431


[0.6063909191604054, 0.74311924]

In [40]:
model7 = run(epochs=5, embed_dim=100, dropout=0.6, dense_layer_dims=[64, 32], num_filters=[2,2,2], kernel_sizes=[2,4,6])

{'epochs': 5, 'embed_dim': 100, 'num_filters': [2, 2, 2], 'kernel_sizes': [2, 4, 6], 'dense_layer_dims': [64, 32], 'dropout': 0.6}
Epoch 1/5
6920/6920 [==============================] - 9s 1ms/sample - loss: 0.6703 - acc: 0.5788
Epoch 2/5
6920/6920 [==============================] - 9s 1ms/sample - loss: 0.5063 - acc: 0.7633
Epoch 3/5
6920/6920 [==============================] - 9s 1ms/sample - loss: 0.3524 - acc: 0.8578
Epoch 4/5
6920/6920 [==============================] - 9s 1ms/sample - loss: 0.2576 - acc: 0.8974
Epoch 5/5
6920/6920 [==============================] - 9s 1ms/sample - loss: 0.1851 - acc: 0.9299


In [41]:
model7.evaluate(dev_x, dev_y)

872/872 [==============================] - 0s 377us/sample - loss: 0.5923 - acc: 0.7603


[0.5922778399165617, 0.7603211]

In [42]:
model8 = run(epochs=5, embed_dim=100, dropout=0.8, dense_layer_dims=[128, 64], num_filters=[2,2,2], kernel_sizes=[2,4,6])

{'epochs': 5, 'embed_dim': 100, 'num_filters': [2, 2, 2], 'kernel_sizes': [2, 4, 6], 'dense_layer_dims': [128, 64], 'dropout': 0.8}
Epoch 1/5
6920/6920 [==============================] - 9s 1ms/sample - loss: 0.6724 - acc: 0.5766
Epoch 2/5
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.5126 - acc: 0.7538
Epoch 3/5
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.3353 - acc: 0.8568
Epoch 4/5
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.2222 - acc: 0.9123
Epoch 5/5
6920/6920 [==============================] - 8s 1ms/sample - loss: 0.1730 - acc: 0.9315


In [43]:
model8.evaluate(dev_x, dev_y)

872/872 [==============================] - 0s 407us/sample - loss: 0.5644 - acc: 0.7638


[0.5644018691067302, 0.76376146]

In [47]:
# exploring top models
model5.evaluate(test_x, test_y)
model7.evaluate(test_x, test_y)
model8.evaluate(test_x, test_y)

1821/1821 [==============================] - 0s 115us/sample - loss: 0.5430 - acc: 0.7611


[0.5429550334688467, 0.76112026]

In [48]:
model9 = run(epochs=5, embed_dim=25, dropout=0.8, dense_layer_dims=[128, 64], num_filters=[2,2,2], kernel_sizes=[2,4,6])

{'epochs': 5, 'embed_dim': 25, 'num_filters': [2, 2, 2], 'kernel_sizes': [2, 4, 6], 'dense_layer_dims': [128, 64], 'dropout': 0.8}
Epoch 1/5
6920/6920 [==============================] - 4s 546us/sample - loss: 0.6855 - acc: 0.5501
Epoch 2/5
6920/6920 [==============================] - 3s 462us/sample - loss: 0.5529 - acc: 0.7241
Epoch 3/5
6920/6920 [==============================] - 3s 481us/sample - loss: 0.3894 - acc: 0.8331
Epoch 4/5
6920/6920 [==============================] - 3s 482us/sample - loss: 0.2937 - acc: 0.8822
Epoch 5/5
6920/6920 [==============================] - 3s 465us/sample - loss: 0.2172 - acc: 0.9147


In [49]:
model9.evaluate(dev_x, dev_y)

872/872 [==============================] - 0s 404us/sample - loss: 0.5985 - acc: 0.7511


[0.5985016472842715, 0.7511468]